## Recursive neural networks for word embedding

As my first foray into recursive neural networks for natural language processing I've decided to set up a fairly vanilla model in theano and see what happens when I give it 5 million sentences and almost 700K training sentences. I don't have any special hardware so training will be slow, but let's see how far I can get. 

My main motivation is the paper by [Mikolov, Yih, and Zweig](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/rvecs.pdf) of Microsoft Research. In that paper they demonstrate that the continuous word embedding encoded in a trained RNN model captures structural features of the given language (English). For example, in the embedding space the vector direction from 'man' to 'woman' is close to the vector direction from 'uncle' to 'aunt'. No explanation for this phenomenon is given, and it is frankly rather bewildering. This type of language 'structure encoding' is one of the magical aspects of the well-known [word2vec](https://arxiv.org/pdf/1301.3781.pdf) embedding. The model in [Mikolov, Yih, and Zweig](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/rvecs.pdf) looks to be fairly straight-forward to set-up so it should make a good starting point for me.

## The model

In [Mikolov, Yih, and Zweig](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/rvecs.pdf) the model is surprisingly simple:

$$ \mathbf{h}_t = f(U\mathbf{x}_t + W\mathbf{h}_{t-1}) $$

and

$$ \mathbf{y}_t = g(V\mathbf{h}_t) $$

where $f$ is the [logistic sigmoid](https://en.wikipedia.org/wiki/Sigmoid_function) and $g$ is the [soft max](https://en.wikipedia.org/wiki/Softmax_function) function. The point is that if the input vectors $\mathbf{x}_t$ are images of vocabulary words under the one-of-$n$ embedding, where the $i$-th vocabulary word is embedded in $\mathbb{R}^n$ as the $i$-th standard basis vector (the vector of all zeros except in the $i$-th coordinate which is one), then the columns of the weight matrix $U$ provide another continuous embedding of the vocabulary, now into $\mathbb{R}^k$ where $k$ is the number of rows of $U$. Namely, the embedding maps the $i$-th vocabulary word to the $i$-th column of $U$.

In order to capture backwards as well as forwards structure I added bidirectionality to the model, and for kicks I put in a bias term: 

$$ \mathbf{h}_t^+ = f(U\mathbf{x}_t + W\mathbf{h}_{t-1}^+ + \mathbf{b}) $$

and

$$ \mathbf{h}_t^- = f(U\mathbf{x}_{m-t} + W\mathbf{h}_{t-1}^- + \mathbf{b}) $$

and

$$ \mathbf{y}_t = g(V(\mathbf{h}_t^+ + \mathbf{h}_t^-)),$$

$m$ is the number of words in the sentence. 

But what are we modeling with this RNN? Under the one-of-$n$ embedding we can think of a given sentence as a vector of integers $\mathbf{s} = (i_1,\ldots,i_m)$. Denote the $i$-th coordinate of a vector $\mathbf{v}$ by $\mathbf{v}[i]$. We are thinking or the $i$-th coordinate of $\mathbf{y}_t$ as the probability that the $i$-th word of the vocabulary occurs in the $t$-th position of the given sentence:

$$ \mathbf{y}_t[i] = P\left(\mathbf{s}[t] = i\,|\,\mathbf{y}_1,\ldots, \widehat{\mathbf{y}_t}, \ldots,\mathbf{y}_m, s_1,\ldots,\widehat{s_t},\ldots,s_m \,;\, U,W,V,b \right). $$

So ideally,

$$\mathbf{y}_t[i] = \left\{ \begin{array}{ll} 1 & \text{if } i = \mathbf{s}[t] \\ 0 & \text{otherwise} \end{array} \right. .$$

Our task then is to find weights $U$, $W$ and $V$ and bias $b$ such that the above equation holds as much as possible. Explicitly our optimization problem is:

$$ \max _{U,W,V,b}\sum_{\mathbf{s}}\sum_t \log \mathbf{y}_t[\mathbf{s}[t]],$$

which is maximum log likelyhood on the probabilities that the $\mathbf{s}[t]$-th word of the vocabulary occurs in the $t$-th position of any given sentence $\mathbf{s}$.